# 4.2 逆確率重み付け法（IPTW）による因果推論

本ファイルは、4.2節の実装です。

4.1節と同じく、テレビCMの広告効果の推定を例に、回帰分析による因果推論を実装します。


## プログラム実行前の設定など

In [0]:
# 乱数のシードを設定
import random
import numpy as np

np.random.seed(1234)
random.seed(1234)


In [0]:
# 使用するパッケージ（ライブラリと関数）を定義
# 標準正規分布の生成用
from numpy.random import *

# グラフの描画用
import matplotlib.pyplot as plt

# SciPy 平均0、分散1に正規化（標準化）関数
import scipy.stats

# シグモイド関数をimport
from scipy.special import expit

# その他
import pandas as pd


## データの作成

In [0]:
# データ数
num_data = 200

# 年齢
x_1 = randint(15, 76, num_data)  # 15から75歳の一様乱数

# 性別（0を女性、1を男性とします）
x_2 = randint(0, 2, num_data)  # 0か1の乱数


## テレビCMを見たかどうか

In [0]:
# ノイズの生成
e_z = randn(num_data)

# シグモイド関数に入れる部分
z_base = x_1 + (1-x_2)*10 - 40 + 5*e_z

# シグモイド関数を計算
z_prob = expit(0.1*z_base)

# テレビCMを見たかどうかの変数（0は見ていない、1は見た）
Z = np.array([])

for i in range(num_data):
    Z_i = np.random.choice(2, size=1, p=[1-z_prob[i], z_prob[i]])[0]
    Z = np.append(Z, Z_i)


## 購入量Yを作成

In [0]:
# ノイズの生成
e_y = randn(num_data)

Y = -x_1 + 30*x_2 + 10*Z + 80 + 10*e_y


## データをまとめた表を作成し、平均値を比べる

In [6]:
df = pd.DataFrame({'年齢': x_1,
                   '性別': x_2,
                   'CMを見た': Z,
                   '購入量': Y,
                   })

df.head()  # 先頭を表示


,年齢,性別,CMを見た,購入量
0,62,0,1.0,24.464285
1,34,0,0.0,45.693411
2,53,1,1.0,64.998281
3,68,1,1.0,47.186898
4,27,1,0.0,100.114260


In [7]:
# 平均値を比べる

print(df[df["CMを見た"] == 1.0].mean())
print("--------")
print(df[df["CMを見た"] == 0.0].mean())


年齢       55.836066
性別        0.483607
CMを見た     1.000000
購入量      49.711478
dtype: float64
--------
年齢       32.141026
性別        0.692308
CMを見た     0.000000
購入量      68.827143
dtype: float64


## ここからが4.1節と異なります。傾向スコアの推定

In [8]:
# scikit-learnからロジスティク回帰をimport
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
from sklearn.linear_model import LogisticRegression

# 説明変数
X = df[["年齢", "性別"]]

# 被説明変数（目的変数）
Z = df["CMを見た"]

# 回帰の実施
reg = LogisticRegression().fit(X,Z)

# 回帰した結果の係数を出力
print("係数beta：", reg.coef_)
print("係数alpha：", reg.intercept_)

係数beta： [[ 0.10562765 -1.38263933]]
係数alpha： [-3.37146523]


### 各人の傾向スコアを求める

In [9]:
Z_pre = reg.predict_proba(X)
print(Z_pre[0:5])  # 5人ほどの結果を見てみる
print("----")
print(Z[0:5])  # 5人ほどの正解


[[0.04002323 0.95997677]
 [0.44525168 0.55474832]
 [0.30065918 0.69934082]
 [0.08101946 0.91898054]
 [0.87013558 0.12986442]]
----
0    1.0
1    0.0
2    1.0
3    1.0
4    0.0
Name: CMを見た, dtype: float64


### 平均処置効果ATEを求める

In [10]:
ATE_i = Y/Z_pre[:, 1]*Z - Y/Z_pre[:, 0]*(1-Z)
ATE = 1/len(Y)*ATE_i.sum()
print("推定したATE", ATE)


推定したATE 8.847476810855458


以上